# Forms

We'll continue working on our Blog application by adding forms so a user can create, edit, or delete any of their blog entries. 

HTML forms are one of the more complicated and error-prone aspects of web development because any time you accept user input, there are security concerns. 

All forms must be properly rendered, validated, and saved to the database.

Writing this code by hand would be time-consuming and difficult, so Django comes with powerful built-in Forms that abstract away much of the difficulty
for us. 

Django also comes with generic editing views for common tasks like displaying, creating, updating, or deleting a form.

### CreateView

To start, update our base template to display a link to a page for entering new blog posts. 

It will take the form <a href="{% url 'post_new' %}"></a> where post_new is the name for our URL.

Your updated file should look as follows: 

```html
<!-- templates/base.html -->
{% load static %}
<html>
<head>
<title>Django blog</title>
<link href="https://fonts.googleapis.com/css?family=\
Source+Sans+Pro:400" rel="stylesheet">

<link href="{% static 'css/base.css’' %}" rel="stylesheet">
</head>
<body>
<div>
<header>
<!-- start new HTML... -->

<div class="nav-left">
    <h1><a href="{% url 'home' %}">Django blog</a></h1>
</div>

<div class="nav-right">
    <a href="{% url 'post_new' %}">+ New Blog Post</a>
</div>

<!-- end new HTML... -->
</header>

{% block content %}

{% endblock content %}
</div>
</body>
</html>
```

Let’s add a new URL for post_new now. 

Import BlogCreateView (which has not been created yet) at the top and then add a URL path for post/new/.

We will give it the URL name post_new to refer to it later in our templates.

```python
# blog/urls.py
from django.urls import path
from .views import BlogListView, BlogDetailView, BlogCreateView # new

urlpatterns = [
    path("post/new/", BlogCreateView.as_view(), name="post_new"), # new
    path("post/<int:pk>/", BlogDetailView.as_view(), name="post_detail"),
    path("", BlogListView.as_view(), name="home"),
]
```

Simple, right? It’s the same URL, views, and template pattern we’ve seen before. Now let’s create our view by importing a generic class-based view called
CreateView at the top and then subclass it to create a new view called BlogCreateView.

```python
# blog/views.py
from django.views.generic import ListView, DetailView
from django.views.generic.edit import CreateView # new
from .models import Post
class BlogListView(ListView):
    model = Post
    template_name = "home.html"

class BlogDetailView(DetailView):
    model = Post
    template_name = "post_detail.html"

class BlogCreateView(CreateView): # new
    model = Post
    template_name = "post_new.html"
    fields = ["title", "author", "body"]

```


Within BlogCreateView, we specify our database model, Post, the name of our template, post_new.html, and explicitly set the database fields we want to expose, which are title, author, and body.

The last step is creating our template, templates/post_new.html. 

Then add the following code:

```html
<!-- templates/post_new.html -->
{% extends "base.html" %}
{% block content %}
    <h1>New post</h1>
    <form action="" method="post">{% csrf_token %}
    {{ form.as_p }}
        <input type="submit" value="Save">
    </form>
{% endblock content %}
```

Let’s break down what we've done:
- on the top line we extended our base template.
- use HTML <form> tags with the POST method since we’re sending data. 
If we were receiving data from a form, for example, in a search box, we would
use GET.
- add a {% csrf_token %} which Django provides to protect our form from cross-site request forgery. You should use it for all your Django forms.
- then, to output our form data use {{ form.as_p }2}, which renders the specified fields within paragraph <p> tags.
- finally, specify an input type of submit and assign the value “Save”.

To view our work, start the server with python manage.py runserver and go to the homepage at http://127.0.0.1:8000/.

Try to create a new blog post and submit it by clicking the “Save” button.

Upon completion, it will **redirect to a detail page athttp://127.0.0.1:8000/post/3/** with the post itself. Success!

### UpdateView

Creating an update form so users can edit blog posts should feel familiar. We’ll again use a built-in Django class-based generic view, UpdateView, and
create the requisite template, URL, and view.

To start, let’s add a new link to , let’s add a post_detail.html so that the option to edit a blog post appears on an individual blog page.


```html
<!-- templates/post_detail.html -->

{% extends "base.html" %}

{% block content %}

<div class="post-entry">

<h2>{{ post.title }}</h2>
<p>{{ post.body }}</p>

</div>

<!-- start new HTML... -->

<a href="{% url 'post_edit' post.pk %}">+ Edit Blog Post</a>

<!-- end new HTML... -->

{% endblock content %}
```

We've added a link using <a href>...</a>and the Django template engine’s {% url ... %} tag. Within it, we've specified the target name of our

'URL, which will be called post_edit, and also passed the parameter needed, which is the primary key of the post: post . pk.

Next, we create the template file for our edit page called templates/post_edit.html and add the following code:

```html
<!-- templates/post_edit.html -->

{% extends "base.html" %}

{% block content %}

<h1>Edit post</h1>
<form action="" method="post">{% csrf_token %}
    {{ form.as_p }}
    <input type="submit" value="Update">
</form> 
{% endblock content %}
```


We again use HTML <form></form> tags, Django’s csrf_token for security, form.as_p to display our form fields with paragraph tags, and finally give it the value “Update” on the submit button.

Now to our view. We must import UpdateView on the second-from-the-top line and then subclass it in our new view BlogUpdateView.

```python
# blog/views.py
from django.views.generic import ListView, DetailView
from django.views.generic.edit import CreateView, UpdateView # new
from .models import Post
class BlogListView(ListView):
    model = Post
    template_name = "home.html"

class BlogDetailView(DetailView):
    model = Post
    template_name = "post_detail.html" 

class BlogCreateView(CreateView):
    model = Post
    template_name = "post_new.html"
    fields = ["title", "author", "body"]

class BlogUpdateView(UpdateView): # new
    model = Post
    template_name = "post_edit.html"
    fields = ["title", "body"]
```



Notice in BlogUpdateView we assume that the author of the post is not changing; we only want the title and body text to be editable, hence ["title", "body" ] but not author as is the case for BlogCreateView.

The final step is to update our urls.py file as follows: add BlogUpdateView at the top and then create the new route to the existing urlpatterns,
giving the new URL a pattern of /post/pk/edit and name of post_edit.


```python
# blog/urls.py
from django.urls import path
from .views import (
    BlogListView,
    BlogDetailView,
    BlogCreateView,
    BlogUpdateView, # new
)

urlpatterns = [
    path("post/new/", BlogCreateView.as_view(), name="post_new"),
    path("post/<int:pk>/", BlogDetailView.as_view(), name="post_detail"),
    path("post/<int:pk>/edit/", BlogUpdateView.as_view(), name="post_edit"), # new
    path("", BlogListView.as_view(), name="home"),
] 
```

At the top, we added our view BlogUpdateView to the list of imported views, then create a new URL pattern for /post/pk/edit and give it the name post_edit.

Now if you click on a blog entry, you'll see our new + Edit Blog Post hyperlink.

### DeleteView

The process for creating a form to delete blog posts is very similar to that for updating a post. 

We’ll use yet another generic class-based view, DeleteView, create the necessary view, URL, and template.


Let’s start by adding a link to delete blog posts on our individual blog page, post_detail.html.

```html 
<!-- templates/post_detail.html -->
{% extends "base.html" %}
{% block content %}
<div class="post-entry">
    <h2>{{ post.title }}</h2>
    <p>{{ post.body }}</p>
</div>
<div>
<!-- start new HTML... -->
    <p><a href="{% url 'post_edit' post.pk %}">+ Edit Blog Post</a></p>
    <p><a href="{% url 'post_delete' post.pk %}">+ Delete Blog Post</a></p>
<!-- end new HTML... -->
</div>
{% endblock content %}
```

Then create a new file for our delete page template. It will be called templates/post_delete.html and contain the following code:


```html
<!-- templates/post_delete.html -->
{% extends "base.html" %}
{% block content %}
<h1>Delete post</h1>
<form action="" method="post">{% csrf_token %}
    <p>Are you sure you want to delete "{{ post.title }}"?</p>
    <input type="submit" value="Confirm">
</form>
{% endblock content %}
```

Note we are using post.tit1e here to display the title of our blog post. 

We could also just use object.title as it too is provided by DetailView.  

Now update the blog/views.py file by importing DeleteView and reverse_lazy at the top and then create a new view that subclasses DeleteView.

```python
# blog/views.py
from django.views.generic import ListView, DetailView
from django.views.generic.edit import CreateView, UpdateView, DeleteView # new
from django.urls import reverse_lazy # new
from .models import Post

    .....
    .....
    .....
    .....

class BlogDeleteView(DeleteView): # new
    model = Post
    template_name = "post_delete.html"
    success_url = reverse_lazy("home")
```

The DeleteView specifies a model which is Post, a template which is post_delete.html, and a third field called success_url. 

What does this do?

Well, after a blog post is deleted, we want to redirect the user to another page which is, in our case, the homepage at home.

You may have noticed we are using reverse_lazy here and not reverse. 
Why is that? 

Both reverse and reverse_lazy perform the same
task: generating a URL based on an input like the URL name. 

The difference is when they are evaluated. reverse executes right away, so when
BlogDeleteView is executed, immediately the model, template_name,and success_url methods are loaded. 

But the success_url needs to find out what the resulting URL path is associated with the URL name “home.” 
It can’t always do that in time. 

That’s why we use reverse_lazy in this
example: it delays the actual call to the URLConf until the moment it is needed, not when our class BlogDeleteView is being evaluated.


The moment BlogDeleteView is called, reverse needs to have the information from the URLconf about what the proper route is for the URL name
“home.” But it might not have that information in time for the success_url.

***https://docs.djangoproject.com/en/4.2/ref/urlresolvers/#reverse-lazy**

CreateView and UpdateView also have redirects. 

Yet, we did not have to specify a success_ur1l because Django
will automatically use **get_absolute_url()** on the model object if available. 

And the only way to know about this trait is to read and remember the
docs, which talk about model forms and success_url. 

Or-the more likely situation-is that an error crops up, and you need to backtrack to sort out this
internal Django behavior.


As a final step, create a URL by importing our view BlogDeleteView and adding a new pattern:

```python
# blog/urls.py
from django.urls import path
from .views import (
    BlogListView,
    BlogDetailView,
    BlogCreateView,
    BlogUpdateView,
    BlogDeleteView, # new
)
urlpatterns = [
    path("post/new/", BlogCreateView.as_view(), name="post_new"),
    path("post/<int:pk>/", BlogDetailView.as_view(), name="post_detail"),
    path("post/<int:pk>/edit/", BlogUpdateView.as_view(),
        name="post_edit"),
    path("post/<int:pk>/delete/", BlogDeleteView.as_view(),
        name="post_delete"), # new
    path("", BlogListView.as_view(), name="home"),
]
```


If you start the server again with the command python manage.py runserver and refresh any individual post page, you'll see our “Delete Blog Post”
link.

### reverse vs. reverse_lazy

Given the provided `views.py`, the line causing a circular import is:

```python
success_url = reverse("home")
```

The reason for this circular import is as follows:

1. When Django starts, it tries to set up the URL configuration.
2. As part of the URL configuration setup, it will attempt to import all views related to the URLs.
3. When importing the `views.py` file, the module-level statement `success_url = reverse("home")` is executed. !! **class-level attributes in Python classes are evaluated when the module is imported** !!
4. The `reverse` function will try to resolve the "home" URL pattern.
5. To resolve the "home" URL, Django needs to look at all URL configurations again.
6. If the "home" URL is related to one of the views in this `views.py` file (or even indirectly related due to including other URL patterns which in turn depend on this `views.py`), it results in a circular import: Django is trying to import `views.py` while it's already in the process of importing it.

Using `reverse_lazy` instead of `reverse` fixes the issue because:

- `reverse_lazy` doesn't immediately resolve the URL when it's defined. Instead, it waits until the URL is actually needed. 
- In the case of class-based views, this means waiting until an instance of the class (e.g., `BlogDeleteView`) is created and the `success_url` is accessed.
- By the time the URL is actually accessed in this manner, the URL configuration has been fully loaded and set up, thus avoiding the circular import.

So, changing the line to:

```python
success_url = reverse_lazy("home")
```

would resolve the circular import issue by deferring the URL resolution until it's absolutely needed.

### Tests

Time for tests to make sure everything works now-and in the future-as expected. We've added new views for create, update, and delete, so that means three new tests:

three new tests:
-def test_post_createview
-def test_post_updateview
-def test_post_deleteview
Update your existing tests. py file with new tests below test_post_detailview as follows.

```python
# blog/tests.py
...
def test_post_createview(self): # new
    response = self.client.post(
    reverse('post_new'),
    {
    "title": "New title",
    "body": "New text",
    "author": self.user.id,
    },
    )

    self.assertEqual(response.status_code, 302)
    self.assertEqual(Post.objects.last().title, "New title")
    self.assertEqual(Post.objects.last().body, "New text")

def test_post_updateview(self): # new
    response = self.client.post(
    reverse("post_edit", args="1"),
    {
    "title": "Updated title",
    "body": "Updated text",
    },
    )
    self.assertEqual(response.status_code, 302)
    self.assertEqual(Post.objects.last().title, "Updated title")
    self.assertEqual(Post.objects.last().body, "Updated text")

def test_post_deleteview(self): # new
    response = self.client.post(reverse('post_delete', args="1")
    {
    "title": "Updated title",
    "body": "Updated text",
    },
    )
    self.assertEqual(response.status_code, 302)
    self.assertEqual(Post.objects.last().title, "Updated title")
    self.assertEqual(Post.objects.last().body, "Updated text")

def test_post_deleteview(self): # new
    response = self.client.post(reverse("post_delete", args="1"))
    self.assertEqual(response.status_code, 302)
```

For test_post_createview, we create a new response and check that the page has a 302 redirect status code and that the 1ast () object created on
our model matches the new response. Then test_post_updateview sees if we can update the initial post created in setUpTestData since that data
is available throughout our entire test class. The last new test, test_post_deleteview, confirms that a 302 redirect occurs when deleting a post.
More tests can always be added later, but at least we have some coverage on all our new functionality. Stop the local web server with Control+c and run
these tests now. They should all pass.

### Conclusion

With a small amount of code, we’ve built a blog application that allows for creating, reading, updating, and deleting blog posts. This core functionality is
known by the acronym CRUD: Create-Read-Update-Delete. The majority of websites in the world consist of this core functionality. While there are multiple
‘ways to achieve this same functionality-we could have used function-based views or written our own class-based views-we’ve demonstrated how little
code it takes in Django to make this happen.

Note, however, a potential security concern: currently any user can update or delete blog entries, not just the creator! Fortunately, Django has built-in
features to restrict access based on permissions, which we’ll cover later in the book.

But for now, our blog application is working, and in the next chapter, we’ll add user accounts so users can sign up, log in, and log out of the web app.

### Bonus Info (if time is left)

Using Django's powerful ModelForm feature, the form is designed to handle both the rendering and data validation for creating or updating articles.

```python
from django import forms
from .models import Article

class ArticleForm(forms.ModelForm):
    class Meta:
        model = Article
        fields = ['title', 'content']
        widgets = {
            'title': forms.TextInput(attrs={'class': 'my-input-class'}),
            'content': forms.Textarea(attrs={'class': 'my-textarea-class'}),
        }

```


  - Inherits from `forms.ModelForm`
  
  - `Meta` Inner Class: Provides configuration details about how the form should be constructed and behave:
    - `model = Article`: Links the form to the `Article` model, making sure the form knows how to save itself to the corresponding database table.
    - `fields = ['title', 'content']`: Specifies the fields from the `Article` model to be included in the form.
    - `widgets`: Customizes the HTML rendering of the form fields:
      - The `title` field will render as an `<input type="text">` element with the CSS class "my-input-class".
      - The `content` field will appear as a `<textarea>` with the CSS class "my-textarea-class".

The Meta inner class in Django models and forms serves as a way to encapsulate configurations or metadata about the main class. By doing so, it keeps the primary class clean and focused on its primary responsibilities while offloading the configuration or metadata details to a contained inner class.

# change author
```python

from django import forms
from django.contrib.auth.models import User
from .models import Post

class PostForm(forms.ModelForm):
    author = forms.ModelChoiceField(
        queryset=User.objects.all(),
        widget=forms.Select(attrs={'class': 'form-control'})
    )

    class Meta:
        model = Post
        fields = ['title', 'body', 'author']
        widgets = {
            'title': forms.TextInput(attrs={'class': 'form-control'}),
            'body': forms.Textarea(attrs={'class': 'form-control'}),
            # Note: 'author' widget is defined outside of Meta, so we don't include it here.
        }

```